In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('data/data_daily_by_station.gz.parquet')
print(df.shape)
df.head()

(171163, 19)


C:\Users\amaguire\AppData\Local\Continuum\anaconda3\envs\ami\lib\site-packages\pyarrow\pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
C:\Users\amaguire\AppData\Local\Continuum\anaconda3\envs\ami\lib\site-packages\pyarrow\pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
C:\Users\amaguire\AppData\Local\Continuum\anaconda3\envs\ami\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,latitide,longitude,rain_max,rain_mean,rain_min,rain_std,rhum_max,rhum_mean,rhum_min,rhum_std,station_number,temp_max,temp_mean,temp_min,temp_std,wdsp_max,wdsp_mean,wdsp_min,wdsp_std
date,,,,,,,,,,,,,,,,,,,
1989-01-01,51.793,-8.244,0.0,0.00,0.0,0.00,97.0,86.33,74.0,7.54,1075,10.0,9.33,8.2,0.64,16.0,11.71,7.0,2.16
1989-01-02,51.793,-8.244,0.5,0.03,0.0,0.11,96.0,84.46,73.0,5.16,1075,9.1,8.86,8.5,0.18,17.0,13.96,7.0,2.03
1989-01-03,51.793,-8.244,1.9,0.22,0.0,0.51,99.0,89.25,75.0,7.04,1075,10.9,10.12,9.2,0.52,30.0,23.96,18.0,3.26
1989-01-04,51.793,-8.244,0.7,0.07,0.0,0.17,86.0,80.25,72.0,4.38,1075,9.2,6.60,5.0,1.42,29.0,17.88,9.0,6.32
1989-01-05,51.793,-8.244,1.2,0.26,0.0,0.36,99.0,91.38,81.0,5.48,1075,12.4,9.65,5.9,2.04,25.0,16.58,7.0,5.83


In [3]:
import s2sphere

r = s2sphere.RegionCoverer()
p1 = s2sphere.LatLng.from_degrees(33, -122)
p2 = s2sphere.LatLng.from_degrees(33.1, -122.1)
cell_ids = r.get_covering(s2sphere.LatLngRect.from_point_pair(p1, p2))
print(cell_ids)

[CellId: 80f0694000000000, CellId: 80f06b4000000000, CellId: 80f06bc000000000, CellId: 80f06d0000000000, CellId: 80f06e4000000000, CellId: 80f06ec000000000, CellId: 80f0724000000000, CellId: 80f072c000000000]


In [8]:
cell = s2sphere.Cell.from_lat_lng(s2sphere.LatLng.from_degrees(33, -122))

In [115]:
cell

Cell: face 4, level 30, orientation 0, id 9291052239093219947

In [121]:
type(cell)

s2sphere.sphere.Cell

In [124]:
type(s2sphere.Cell(cell.id().parent()))

s2sphere.sphere.Cell

In [43]:
import s2sphere

r = s2sphere.RegionCoverer()
p = s2sphere.LatLng.from_degrees(53.4, -7.9)
cell_ids = r.get_covering(s2sphere.LatLngRect.from_point(p))
print(cell_ids)

[CellId: 485c494589ad39f1]


In [62]:
import s2sphere

def s2_latlng_to_polygon(latlng,parent_level=7):
    cell = s2sphere.Cell.from_lat_lng(s2sphere.LatLng.from_degrees(*latlng))
    parent_cell = s2sphere.Cell(cell.id().parent(parent_level))
    polygon_tuples = [(float(x[1]),float(x[0])) for x in [str(s2sphere.LatLng.from_point(parent_cell.get_vertex(k))).replace('LatLng: ','').split(',') for k in range(4)]]
    return polygon_tuples
    
s2_latlng_to_polygon((53.4,-7.9),parent_level=30)

[(-7.899999924726439, 53.3999999888054),
 (-7.899999948305971, 53.40000007014335),
 (-7.900000056757584, 53.400000062940045),
 (-7.900000033178052, 53.39999998160209)]

In [45]:
import s2sphere
cell = s2sphere.Cell.from_lat_lng(s2sphere.LatLng.from_degrees(53.4, -7.9))
n_steps = 30
cell_meta = dict()
curr_cell = cell
for n in range(n_steps):
    curr_cell_uid = curr_cell.id()
    curr_cell_id = curr_cell.id().id()
    curr_cell_level = curr_cell.id().level()
    cell_meta[f'level_{curr_cell_level}'] = {
        'cell_id' : curr_cell_id,
        'cell_uid' : curr_cell_uid,
        'cell_vertex_list' : [(float(x[1]),float(x[0])) for x in [str(s2sphere.LatLng.from_point(curr_cell.get_vertex(k))).replace('LatLng: ','').split(',') for k in range(4)]]
    }
    parent_cell = s2sphere.Cell(curr_cell.id().parent())
    curr_cell = parent_cell
cell_meta

{'level_30': {'cell_id': 5214123031599593969,
  'cell_uid': CellId: 485c494589ad39f1,
  'cell_vertex_list': [(-7.899999924726439, 53.3999999888054),
   (-7.899999948305971, 53.40000007014335),
   (-7.900000056757584, 53.400000062940045),
   (-7.900000033178052, 53.39999998160209)]},
 'level_29': {'cell_id': 5214123031599593972,
  'cell_uid': CellId: 485c494589ad39f4,
  'cell_vertex_list': [(-7.899999924726439, 53.3999999888054),
   (-7.899999971885503, 53.40000015148129),
   (-7.900000188788726, 53.40000013707468),
   (-7.900000141629661, 53.39999997439879)]},
 'level_28': {'cell_id': 5214123031599593968,
  'cell_uid': CellId: 485c494589ad39f0,
  'cell_vertex_list': [(-7.899999660664155, 53.39999984053612),
   (-7.899999754982276, 53.4000001658879),
   (-7.900000188788726, 53.40000013707468),
   (-7.900000094470601, 53.39999981172291)]},
 'level_27': {'cell_id': 5214123031599593920,
  'cell_uid': CellId: 485c494589ad39c0,
  'cell_vertex_list': [(-7.899999566346033, 53.39999951518434),


In [170]:
import s2sphere
lat = 52.809766
lng = -2.088996
cell_id = s2sphere.CellId.from_lat_lng(s2sphere.LatLng.from_degrees(lat,lng)).parent(30)
print (s2sphere.LatLng.from_point(s2sphere.Cell(cell_id).get_center()))
print (s2sphere.LatLng.from_point(s2sphere.Cell(cell_id).get_vertex(0)))
print (s2sphere.LatLng.from_point(s2sphere.Cell(cell_id).get_vertex(1)))
print (s2sphere.LatLng.from_point(s2sphere.Cell(cell_id).get_vertex(2)))
print (s2sphere.LatLng.from_point(s2sphere.Cell(cell_id).get_vertex(3)))

LatLng: 52.80976598419109,-2.0889960238054526
LatLng: 52.80976594422005,-2.0889959719554083
LatLng: 52.80976602587492,-2.0889959781407157
LatLng: 52.809766024162116,-2.0889960756555026
LatLng: 52.80976594250725,-2.0889960694701943


In [12]:
import matplotlib.pyplot as plt
import s2sphere
from shapely.geometry import Polygon
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

%matplotlib inline

proj = cimgt.MapQuestOSM()
plt.figure(figsize=(20,20), dpi=200)
ax = plt.axes(projection=proj.crs)
ax.add_image(proj, 12)
ax.set_extent([-51.411886, -50.922470,
               -30.301314, -29.94364])
region_rect = s2sphere.LatLngRect(
    s2sphere.LatLng.from_degrees(-51.264871, -30.241701),
    s2sphere.LatLng.from_degrees(-51.04618, -30.000003))
coverer = s2sphere.RegionCoverer()
coverer.min_level = 8
coverer.max_level = 15
coverer.max_cells = 500
covering = coverer.get_covering(region_rect)
geoms = []
for cellid in covering:
    new_cell = s2sphere.Cell(cellid)
    vertices = []
    for i in range(0, 4):
        vertex = new_cell.get_vertex(i)
        vertex_tmp = str(s2sphere.LatLng.from_point(vertex))
        vertex_tmp_lat = float(vertex_tmp.split(',')[0].split(':')[-1])
        vertex_tmp_long = float(vertex_tmp.split(',')[-1])
        latlng = s2sphere.LatLng(vertex_tmp_lat,vertex_tmp_long)
        #latlng = s2sphere.LatLng.from_point(vertex)
        vertices.append((latlng.lat().degrees(),
                         latlng.lng().degrees()))
    geo = Polygon(vertices)
    geoms.append(geo)
print("Total Geometries: {}".format(len(geoms)))
    
ax.add_geometries(geoms, ccrs.PlateCarree(), facecolor='coral',
                  edgecolor='black', alpha=0.4)
plt.show()

TypeError: 'float' object is not callable

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000002E776F5E8C8> (for post_execute):


ValueError: A non-empty list of tiles should be provided to merge.

ValueError: A non-empty list of tiles should be provided to merge.

<Figure size 4000x4000 with 1 Axes>

In [134]:
cell.get_rect_bound()

LatLngRect: LineInterval: (0.5759586518954118, 0.5759586536725844), SphereInterval: (-2.129301688541844, -2.1293016870276924)

In [157]:
xx = s2sphere.LatLng.from_point(cell.get_vertex(1))
dir(xx)
str(xx)
#dir(xx.lat)
#xx._LatLng__coords

'LatLng: 32.99999992765168,-122.00000006352735'

In [158]:
[str(s2sphere.LatLng.from_point(cell.get_vertex(k))) for k in range(4)]

['LatLng: 33.00000000471436,-122.00000006352735',
 'LatLng: 32.99999992765168,-122.00000006352735',
 'LatLng: 32.999999952413425,-121.99999997677291',
 'LatLng: 33.00000002947611,-121.99999997677291']

In [135]:
for k in range(4):
    print(s2sphere.LatLng.from_point(cell.get_vertex(k)))

LatLng: 33.00000000471436,-122.00000006352735
LatLng: 32.99999992765168,-122.00000006352735
LatLng: 32.999999952413425,-121.99999997677291
LatLng: 33.00000002947611,-121.99999997677291


In [10]:
level12_cell = s2sphere.Cell(cell.id().parent(12))

In [15]:
level12_cell.id

<bound method Cell.id of Cell: face 4, level 12, orientation 0, id 9291052268944097280>

In [1]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import numpy as np


# Make the figure
fig = plt.figure()
ax = fig.add_subplot(111)

# Easiest way to make a basemap is to use the cylidrical projection and 
# define the bottom left lat/lon and top right lat/lon corners

# Map of Utah
bot_left_lat  =36.5
bot_left_lon  =-114.5
top_right_lat =42.5
top_right_lon = -108.5

# create the map object, m
m = Basemap(resolution='i', projection='cyl', \
    llcrnrlon=bot_left_lon, llcrnrlat=bot_left_lat, \
    urcrnrlon=top_right_lon, urcrnrlat=top_right_lat)

# Note: You can define the resolution of the map you just created. Higher 
# resolutions take longer to create.
#    'c' - crude
#    'l' - low
#    'i' - intermediate
#    'h' - high
#    'f' - full

# Draw some map elements on the map
m.drawcoastlines()
m.drawstates()
m.drawcountries()
m.drawrivers(color='blue')

# Drawing ArcGIS Basemap (only works with cylc projections??)
# Examples of what each map looks like can be found here:
# http://kbkb-wx-python.blogspot.com/2016/04/python-basemap-background-image-from.html
maps = ['ESRI_Imagery_World_2D',    # 0
        'ESRI_StreetMap_World_2D',  # 1
        'NatGeo_World_Map',         # 2
        'NGS_Topo_US_2D',           # 3
        'Ocean_Basemap',            # 4
        'USA_Topo_Maps',            # 5
        'World_Imagery',            # 6
        'World_Physical_Map',       # 7
        'World_Shaded_Relief',      # 8
        'World_Street_Map',         # 9
        'World_Terrain_Base',       # 10
        'World_Topo_Map'            # 11
        ]
print("drawing image from arcGIS server...")
m.arcgisimage(service=maps[8], xpixels=1000, verbose=False)
print("...finished")

# Plot a scatter point at WBB on the map object
lon = -111.85
lat = 40.77
m.scatter(lon,lat,c='r',s=150)

# Plot some wind barbs
lons = np.arange(-115,-100,.5)
lats = np.arange(33,48,.5)
u = np.arange(-5,10,.5)
v = np.arange(5,20,.5)
m.barbs(lons, lats, u, v, color='fuchsia')

# Plot line between two points
# (can also use greatcircle function to be more accurate)
x = [-110, -112]
y = [40, 42]
m.plot(x, y, color='navy', lw=5)

# Fill two polygon shapes
patches = []
homeplate = np.array([[-114,38],[-113,37],[-112,38],[-112,40],[-114,40]])
patches.append(Polygon(homeplate))
triangle = np.array([[-111,38],[-110,37],[-110,42]])
patches.append(Polygon(triangle))
ax.add_collection(PatchCollection(patches, facecolor='lightgreen', edgecolor='k', linewidths=1.5))

# Plot shapefiles: see here: http://basemaptutorial.readthedocs.io/en/latest/shapefile.html

# Plot contours
# m.contour(lons2D, lats2D, values2D)  # contour lines
# m.contourf(lons2D, lats2D, values2D) # contour color filled, can specify a cmap

# Plot gridded data
# m.pcolormesh(lons2D, lats2D, values2D) # can specify a cmap

# Add plot title and other plot elements the normal way
plt.title('Map of Utah Basemap Example')
plt.xlabel('this is the x label')

plt.show()

KeyError: 'PROJ_LIB'